In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
# python ./Tools/fall/src/decrypt_unified.py ./Tools/_3_4_5_k\=2/iscas85/PowerRun/c7552_encrypted.bench

import subprocess


def run_fall_attack(file_path: str) -> str:
    """Runs fall attack and returns the logs of the attack"""
    # Run fall attack
    python_interpreter = "python3"
    script_path = "./../Tools/fall/src/decrypt_unified.py"
    args = [python_interpreter, script_path, file_path, "--alg", "3"]
    result = subprocess.run(
        args,
        capture_output=True,
        text=True,
    )
    # Return the logs of the attack
    return result.stdout + result.stderr


# python ./Tools/fall/src/decrypt_unified.py ./Tools/_3_4_5_k\=2/iscas85/PowerRun/c7552_encrypted.bench
o = run_fall_attack("./../Tools/_3_4_5_k=2/iscas85/PowerRun/c7552_encrypted.bench")
print(o)

In [ ]:
# Need to convert all the .bench files dff into inputs and outputs
from typing import Dict
from BenchParser import BenchParser
from BenchFileWriter import BenchFileWriter
import os


def convert_all_dffs2io(file_path: str, desired_output_path: str = "./ITC99/"):
    print(f"Converting {file_path} to io\n in the location {desired_output_path}")
    bench_file = BenchParser(file_path)
    file_name = file_path.split("/")[-1]
    # print(bench_file.gates)
    new_gates: Dict[str, tuple[str, tuple[str, str]]] = {}
    for g_name, (g_type, g_inputs) in bench_file.gates.items():
        if g_type.lower() == "dff":
            # print(g_name, g_inputs)
            # add the gate to the inputs, outputs and remove it from the gates
            bench_file.inputs.append(g_name)
            for i in g_inputs:
                if i in bench_file.outputs:
                    bench_file.outputs.append(i)
        else:
            new_gates[g_name] = (g_type, g_inputs)
    bench_file.gates = new_gates
    output_file_path = f"""{desired_output_path}{file_name.replace(".bench", "_io.bench")}"""
    bench_file_writer = BenchFileWriter(
        path=output_file_path,
        gates=bench_file.gates,
        inputs=bench_file.inputs,
        outputs=bench_file.outputs,
        flip_flops={}, 
        locking_info=[]
    )
    bench_file_writer.write()
    return output_file_path


In [ ]:
import os


ITC99_path = "../test_runs/ITC99_KeySize=InputSize__RANEKC2/ITC99/"
for file in os.listdir(ITC99_path):
	if file.endswith(".bench"):
		convert_all_dffs2io(f"{ITC99_path}{file}", "./tff/ITC99/")
		# RUn the fall attack
		o = run_fall_attack(f"./tff/ITC99/{file.replace('.bench', '_io.bench')}")
		print(o)

In [ ]:
iscas_path = "../test_runs/Structural_keySize=Input_RAIN/iscas89/"
def convert_and_run_folder(path:str, output:str = "./tff/ITC99/"):
	for file in os.listdir(path):
		if file.endswith(".bench"):
			convert_all_dffs2io(f"{path}{file}", output)
			# RUn the fall attack
			o = run_fall_attack(f"{output}{file.replace('.bench', '_io.bench')}")
			print(o)

convert_and_run_folder(iscas_path, "./tff/iscas89/")